In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import cv2

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPool2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
X_train = pd.read_csv('../input/ahcd1/csvTrainImages 13440x1024.csv')
y_train = pd.read_csv('../input/ahcd1/csvTrainLabel 13440x1.csv')

X_test = pd.read_csv('../input/ahcd1/csvTestImages 3360x1024.csv')
y_test = pd.read_csv('../input/ahcd1/csvTestLabel 3360x1.csv')

In [ ]:
test_data = pd.read_csv('../input/ahcd1/csvTestImages 3360x1024.csv')
test_label = pd.read_csv('../input/ahcd1/csvTestLabel 3360x1.csv')

In [ ]:
X_train = X_train.iloc[:,:].values.astype('float32')
y_train = y_train.iloc[:,:].values.astype('int32')-1 # fix the index in dataset

X_test = X_test.iloc[:,:].values.astype('float32')
y_test = y_test.iloc[:,:].values.astype('int32')-1

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 32, 32)
X_test = X_test.reshape(X_test.shape[0], 32, 32)

In [ ]:
print("Train images shape: ", X_train.shape)
print("Train labels shape: ", y_train.shape)

print("Test images shape: ", X_test.shape)
print("Test labels shape: ", y_test.shape)

In [ ]:
plt.figure(figsize=(8,8))

random_num = np.random.randint(0, 13439, size = 16)
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_train[random_num[i]].reshape(32,32).T,"gray")
    plt.xlabel(int(y_train[random_num[i]][0])+1)
plt.show()

In [ ]:
# normalize images
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
n_classes = len(np.unique(y_train))

In [ ]:
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [ ]:
X_train = X_train.reshape([-1, 32, 32, 1])
X_test = X_test.reshape([-1, 32, 32, 1])

# Model

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=5, padding='same', activation='relu', input_shape=(32,32,1)))
model.add(Conv2D(filters=32, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(256, input_dim = 1024, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

# print model
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [ ]:
data_gen = ImageDataGenerator(horizontal_flip = True,
                              width_shift_range = 0.1, 
                              height_shift_range = 0.1)

In [ ]:
data_gen.fit(X_train)

In [ ]:
    hist = model.fit_generator(data_gen.flow(X_train, y_train, batch_size = 128),
                               epochs=50, steps_per_epoch = X_train.shape[0]//128, 
                               validation_data=(X_test, y_test),
                               verbose=1)

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()